In [2]:
import torch
import numpy as np
from torch import linalg as LA
from torch_sym3eig import Sym3Eig
from data.io import readRaw, ReadScalars, ReadTensors

# torch.set_default_dtype(torch.float64)

use_cuda = 0
use_brain = 1
torchdeviceId = torch.device('cuda:0') if use_cuda else 'cpu'
torchdtype = torch.double

dim = 3
if use_brain:
  cases = []
  cases.append('103818')
  cases.append('105923')
  out_tensors = []
  masks = []
  for run_case in cases:
    subj = run_case[:6]
    outroot='/usr/sci/projects/HCP/Kris/NSFCRCNS/TestResults/working_3d_python/simulation_results/'
    outdir = f'{outroot}{run_case}/mineval_0.005_n_3000_s_1.5/'
    indir = '/usr/sci/projects/HCP/Kris/NSFCRCNS/prepped_data/' + subj + '/'

    tens_file = 'scaled_tensors.nhdr'
    out_tens = ReadTensors(outdir + tens_file)
    out_tensors.append(out_tens)
    mask_file = 'filt_mask.nhdr'
    mask = ReadScalars(outdir + mask_file)
    masks.append(mask)
    
  tens_0 = np.zeros((out_tensors[0].shape[0],out_tensors[0].shape[1],out_tensors[0].shape[2],3,3))
  tens_1 = np.zeros((out_tensors[1].shape[0],out_tensors[1].shape[1],out_tensors[1].shape[2],3,3))
  for xx in range(tens_0.shape[0]):
    for yy in range(tens_0.shape[1]):
      for zz in range(tens_0.shape[2]):
        if masks[0][xx,yy,zz]:
          tens_0[xx,yy,zz,0,0] = out_tensors[0][xx,yy,zz,0]
          tens_0[xx,yy,zz,0,1] = out_tensors[0][xx,yy,zz,1]
          tens_0[xx,yy,zz,1,0] = out_tensors[0][xx,yy,zz,1]
          tens_0[xx,yy,zz,0,2] = out_tensors[0][xx,yy,zz,2]
          tens_0[xx,yy,zz,2,0] = out_tensors[0][xx,yy,zz,2]
          tens_0[xx,yy,zz,1,1] = out_tensors[0][xx,yy,zz,3]
          tens_0[xx,yy,zz,1,2] = out_tensors[0][xx,yy,zz,4]
          tens_0[xx,yy,zz,2,1] = out_tensors[0][xx,yy,zz,4]
          tens_0[xx,yy,zz,2,2] = out_tensors[0][xx,yy,zz,5]
        else:
          tens_0[xx,yy,zz,0,0] = 1
          tens_0[xx,yy,zz,1,1] = 1
          tens_0[xx,yy,zz,2,2] = 1
        if masks[1][xx,yy,zz]:
          tens_1[xx,yy,zz,0,0] = out_tensors[1][xx,yy,zz,0]
          tens_1[xx,yy,zz,0,1] = out_tensors[1][xx,yy,zz,1]
          tens_1[xx,yy,zz,1,0] = out_tensors[1][xx,yy,zz,1]
          tens_1[xx,yy,zz,0,2] = out_tensors[1][xx,yy,zz,2]
          tens_1[xx,yy,zz,2,0] = out_tensors[1][xx,yy,zz,2]
          tens_1[xx,yy,zz,1,1] = out_tensors[1][xx,yy,zz,3]
          tens_1[xx,yy,zz,1,2] = out_tensors[1][xx,yy,zz,4]
          tens_1[xx,yy,zz,2,1] = out_tensors[1][xx,yy,zz,4]
          tens_1[xx,yy,zz,2,2] = out_tensors[1][xx,yy,zz,5]
        else:
          tens_1[xx,yy,zz,0,0] = 1
          tens_1[xx,yy,zz,1,1] = 1
          tens_1[xx,yy,zz,2,2] = 1
        

  if use_cuda:
    g0 = torch.inverse(torch.from_numpy(tens_0).cuda().float()).contiguous()
    g1 = torch.inverse(torch.from_numpy(tens_1).cuda().float()).contiguous()

  else:
    g0 = torch.inverse(torch.from_numpy(tens_0).float()).contiguous()
    g1 = torch.inverse(torch.from_numpy(tens_1).float()).contiguous()
else:
  N1, N2, N3 = 100, 100, 100

  Gr = torch.randn(2,N1,N2,N3,dim,dim,dtype=torchdtype, device=torchdeviceId)
  G = torch.einsum("...ks,...ts->...kt",[Gr, Gr])
  g0 = G[0]
  g1 = G[1]

In [3]:
g0.size(), g1.size()

(torch.Size([145, 174, 145, 3, 3]), torch.Size([145, 174, 145, 3, 3]))

In [4]:
# check if g0 and g1 are positive definite symmetric

print(torch.norm(g0 - torch.einsum("...ij->...ji",[g0])), torch.min(torch.det(g0)))
print(torch.norm(g1 - torch.einsum("...ij->...ji",[g1])), torch.min(torch.det(g1)))


tensor(1.3359e-05) tensor(0.0022)
tensor(1.1630e-05) tensor(0.0014)


In [5]:
# print out the nonPDS matrix and the determinant
if torch.min(torch.det(g0))<0:
    g = g0
    detg = torch.det(g)
    I1,I2,I3 = (detg0==torch.min(detg)).nonzero().reshape(-1)
    print('g0 is not PD')
    print(g0[I1,I2,I3])
    print('The det is', g0[I1,I2,I3].det().item())
elif torch.min(torch.det(g1))<0:
    g = g1
    detg = torch.det(g)
    I1,I2,I3 = (detg0==torch.min(detg)).nonzero().reshape(-1)
    print('g1 is not PD')
    print(g1[I1,I2,I3])
    print('The det is',g1[I1,I2,I3].det())

In [6]:
A, B = g0, g1

In [7]:
%%time
G = LA.cholesky(A)
G_inv = torch.inverse(G)
W = torch.einsum("...ik,...ks,...js->...ij",[G_inv, B, G_inv])

CPU times: user 6.26 s, sys: 1.91 s, total: 8.17 s
Wall time: 1.34 s


In [8]:
print(W.device)
print(W.is_contiguous())


cpu
True


In [20]:
#W = torch.from_numpy(ReadScalars('/home/sci/hdai/Shared/W.nhdr')).permute((2,0,1))

In [25]:
#W = WFull[83613]
W.shape

torch.Size([410000, 3, 3])

### Method 1: Use torch.symeig

In [9]:
%%time

Lamb1,Lamb1v = torch.symeig(W, eigenvectors=True)
trKsquare1 = torch.sum(torch.log(Lamb1)**2,(-1))
# Cpu time 1.67s
# GPU time 22 min

CPU times: user 4.12 s, sys: 78.5 ms, total: 4.2 s
Wall time: 3.55 s


/home/sci/kris/Software/py3_venv_nsfcrcns/lib64/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: torch.symeig is deprecated in favor of torch.linalg.eigh and will be removed in a future PyTorch release.
The default behavior has changed from using the upper triangular portion of the matrix by default to using the lower triangular portion.
L, _ = torch.symeig(A, upper=upper)
should be replaced with
L = torch.linalg.eigvalsh(A, UPLO='U' if upper else 'L')
and
L, V = torch.symeig(A, eigenvectors=True)
should be replaced with
L, V = torch.linalg.eigh(A, UPLO='U' if upper else 'L') (Triggered internally at  ../aten/src/ATen/native/BatchLinearAlgebra.cpp:2500.)
  """Entry point for launching an IPython kernel.


In [10]:
W.shape

torch.Size([145, 174, 145, 3, 3])

In [111]:
%%time
W_max = torch.max(W.reshape((*W.shape[0:3],9)),3)[0]
scaled_W = torch.einsum('...ij,...->...ij',W, 1.0 / W_max)
Lamb1Alt = torch.einsum('...,...i->...i', W_max, torch.symeig(scaled_W, eigenvectors=True)[0])
trKsquare1Alt = torch.sum(torch.log(Lamb1Alt)**2,(-1))

CPU times: user 3.85 s, sys: 21 ms, total: 3.87 s
Wall time: 3.43 s


### Method 2: Use sym3eig

In [11]:
%%time
Lamb2,Lamb2v = Sym3Eig.apply(W.reshape((-1,3,3)))
#Lamb2r = Sym3Eig.apply(W.reshape((-1,3,3)))
#if use_brain:
#  #Lamb2 = torch.abs(Lamb2r[0].reshape((145,174,145,3)))
#  Lamb2 = Lamb2r[0].reshape((145,174,145,3))
#else:
#  Lamb2 = torch.abs(Lamb2r[0].reshape((100,100,100,3)))
#s3v = s3vr.reshape((145,174,145,3,3))
trKsquare2 = torch.sum(torch.log(Lamb2)**2,(-1))
# CPU time: 203 ms
# GPU time: 

CPU times: user 849 ms, sys: 127 ms, total: 976 ms
Wall time: 197 ms


In [16]:
print(Lamb1[0,0,0,:],Lamb1v[0,0,0,:,:])
print(Lamb2.reshape((145,174,145,3))[0,0,0,:],Lamb2v.reshape((145,174,145,3,3))[0,0,0,:,:])

tensor([1., 1., 1.]) tensor([[1., -0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]])
tensor([1., 1., 1.]) tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])


### Method3: use torchvectorized vlinalg

In [14]:
from torchvectorized import vlinalg as TV

In [10]:
W.shape

torch.Size([100, 100, 100, 3, 3])

In [177]:
def checkMag(lamb,vec):
  print(lamb.shape, vec.shape)
  #lambmax = torch.max(torch.stack((torch.max(lamb*lamb,-1)[0],torch.max(lamb,-1)[0])),0)[0]
  lambmax = torch.max(lamb*lamb,-1)[0]
  #nrm = (LA.norm(vec,dim=len(vec.shape)-1) **2)
  #thresh = 2**8 * 1e-9 * (lambmax**2)
  #nrm = vec[:,:,:,:,0] * vec[:,:,:,:,0] + vec[:,:,:,:,1] * vec[:,:,:,:,1] + vec[:,:,:,:,2] * vec[:,:,:,:,2]
  #nrm = torch.sqrt(vec[:,:,:,:,0] * vec[:,:,:,:,0] + vec[:,:,:,:,1] * vec[:,:,:,:,1] + vec[:,:,:,:,2] * vec[:,:,:,:,2]).to(lamb.device)
  nrm = torch.sqrt(vec[:,:,:,0,:] * vec[:,:,:,0,:] + vec[:,:,:,1,:] * vec[:,:,:,1,:] + vec[:,:,:,2,:] * vec[:,:,:,2,:]).to(lamb.device)
  thresh = 2**4 * 1e-9 * lambmax
  print(lambmax.device,nrm.device,thresh.device,vec.device)
  #return((torch.where(nrm[:,:,:,0] <= thresh), torch.where(nrm[:,:,:,1] <= thresh), torch.where(nrm[:,:,:,2] <= thresh)))
  #return((torch.where(nrm[:,:,:,0] <= thresh), torch.where(nrm[:,:,:,1] <= thresh), torch.where(nrm[:,:,:,2] <= thresh)))
  idx1 = torch.where(nrm[:,:,:,0] <= thresh)
  idx2 = torch.where(nrm[:,:,:,1] <= thresh)
  idx3 = torch.where(nrm[:,:,:,2] <= thresh)
  idx = (torch.cat((idx1[0], idx2[0], idx3[0])), 
         torch.cat((idx1[1], idx2[1], idx3[1])), 
         torch.cat((idx1[2], idx2[2], idx3[2])))
  return(idx)
                    
def checkMag2(lamb):
  #lambmax = torch.max(torch.stack((torch.max(lamb*lamb,-1)[0],torch.max(lamb,-1)[0])),0)[0]
  lambdiff = lamb[:,:,:,2] / lamb[:,:,:,0]
  return(torch.where(lamb[:,:,:,0] < 1)) # 1e-3: error 0.008, 1e-1: error 0.004, 1: error 0.0019
def checkMag3(lamb,A):
    # From hybrid method at http://www.mpi-hd.mpg.de/personalhomes/globes/3x3/
    # as described here: https://arxiv.org/pdf/physics/0610206.pdf
    abslamb = torch.abs(lamb)
    t = abslamb[:,:,:,2]
    idx = torch.where(abslamb[:,:,:,1] > t)
    print(len(idx[0]))
    t[idx[0],idx[1],idx[2]] = abslamb[idx[0],idx[1],idx[2],1]
    idx = torch.where(abslamb[:,:,:,0] > t)
    print(len(idx[0]))
    t[idx[0],idx[1],idx[2]] = abslamb[idx[0],idx[1],idx[2],0]
    
    u = t
    idx = torch.where(t>=1.0)
    u[idx[0],idx[1],idx[2]] = t[idx[0],idx[1],idx[2]] * t[idx[0],idx[1],idx[2]]
    sqru = u*u
    error = 256*1e-9*sqru
    
    q01 = A[:,:,:,0,1] * A[:,:,:,1,2] - A[:,:,:,0,2] * A[:,:,:,1,1]
    q11 = A[:,:,:,0,2] * A[:,:,:,0,1] - A[:,:,:,1,2] * A[:,:,:,0,0]
    #q21 = A[:,:,:,0,1] * A[:,:,:,0,1]
    q00 = q01 + A[:,:,:,0,2] * lamb[:,:,:,2]
    q10 = q11 + A[:,:,:,1,2] * lamb[:,:,:,2]
    q20 = (A[:,:,:,0,0] - lamb[:,:,:,2]) * (A[:,:,:,1,1] - lamb[:,:,:,2]) - A[:,:,:,0,1] * A[:,:,:,0,1]
    nrm = q00*q00 + q10*q10 + q20*q20

    idx0 = torch.where(nrm <= error)
    
    # Version 2 of this check includes the following to produce idx1:
    
    # Following 3 lines only needed for computing 3rd eigenvector
    #q00_2 = q00 * nrm
    #q10_2 = q10 * nrm
    #q20_2 = q20 * nrm
    
    q01_2 = q01 + A[:,:,:,0,2] * lamb[:,:,:,1]
    q11_2 = q11 + A[:,:,:,1,2] * lamb[:,:,:,1]
    q21 = (A[:,:,:,0,0] - lamb[:,:,:,1]) * (A[:,:,:,1,1] - lamb[:,:,:,1]) - A[:,:,:,0,1] * A[:,:,:,0,1]
    nrm_2 = q01_2*q01_2 + q11_2*q11_2 + q21*q21
    
    idx1 = torch.where(nrm_2 <= error)
    
    idx = (torch.cat((idx0[0], idx1[0])), 
           torch.cat((idx0[1], idx1[1])), 
           torch.cat((idx0[2], idx1[2])))
 
    return(idx)   
    ##nrm = torch.sqrt(q00*q00 + q10*q10 + q20*q20)
    #return(torch.where(nrm <= 256*1e-9*sqru))
    ##return(torch.where(nrm <= 16*1e-4*u))
def checkMag4(lamb,vec):
  print(lamb.shape, vec.shape)
  #lambmax = torch.max(torch.stack((torch.max(lamb*lamb,-1)[0],torch.max(lamb,-1)[0])),0)[0]
  lambmax = torch.max(lamb,dim=-1)[0]
  u = lambmax
  idx = torch.where(lambmax>=1.0)
  u[idx[0],idx[1],idx[2]] = lambmax[idx[0],idx[1],idx[2]]*lambmax[idx[0],idx[1],idx[2]]
  sqru = u*u
  error = 256*1e-9*sqru 

  #nrm = (vec[:,:,:,:,0] * vec[:,:,:,:,0] + vec[:,:,:,:,1] * vec[:,:,:,:,1] + vec[:,:,:,:,2] * vec[:,:,:,:,2]).to(lamb.device)
  nrm = (vec[:,:,:,0,:] * vec[:,:,:,0,:] + vec[:,:,:,1,:] * vec[:,:,:,1,:] + vec[:,:,:,2,:] * vec[:,:,:,2,:]).to(lamb.device)
  #return((torch.where(nrm[:,:,:,0] <= thresh), torch.where(nrm[:,:,:,1] <= thresh), torch.where(nrm[:,:,:,2] <= thresh)))
  #return((torch.where(nrm[:,:,:,0] <= thresh), torch.where(nrm[:,:,:,1] <= thresh), torch.where(nrm[:,:,:,2] <= thresh)))
  # Do we need to check 3rd eigenvector?
  idx1 = torch.where(nrm[:,:,:,0] <= error)
  idx2 = torch.where(nrm[:,:,:,1] <= error)
  idx3 = torch.where(nrm[:,:,:,2] <= error)
  print(len(idx1[0]),len(idx2[0]),len(idx3[0]))
  idx = (torch.cat((idx1[0], idx2[0], idx3[0])), 
         torch.cat((idx1[1], idx2[1], idx3[1])), 
         torch.cat((idx1[2], idx2[2], idx3[2])))
  return(idx)
                    
    

In [52]:
#W = torch.einsum("...ik,...ks,...js->...ij",[G_inv, B, G_inv])
print(W.shape)
print(W.shape[0]*W.shape[1]*W.shape[2])
print(W.permute((3,4,0,1,2)).shape)
print(W.permute((3,4,0,1,2)).reshape((1,9,100,100,100)))

torch.Size([100, 100, 100, 3, 3])
1000000
torch.Size([3, 3, 100, 100, 100])
tensor([[[[[ 8.5207e-01,  2.2181e-01,  3.0813e+00,  ...,  6.0611e+00,
             7.1486e-01,  1.3881e+00],
           [ 2.0328e+00,  1.1288e-01,  2.4111e+00,  ...,  2.4533e+00,
             2.1193e+00,  3.5957e+00],
           [ 7.6886e-01,  3.8845e+00,  3.1359e+00,  ...,  2.8681e-01,
             1.7287e-01,  7.0561e-01],
           ...,
           [ 2.3204e-01,  4.0186e-01,  4.2897e+00,  ...,  1.6221e+01,
             7.0160e-01,  3.1719e+00],
           [ 2.1442e+00,  1.7903e+01,  2.0547e-01,  ...,  1.2468e+00,
             3.5853e+00,  1.7907e+01],
           [ 5.6605e-01,  1.8817e-01,  9.3034e-02,  ...,  2.0682e+00,
             1.8257e+00,  2.8736e-01]],

          [[ 4.4475e-01,  1.6184e-01,  2.6065e-01,  ...,  4.6684e+00,
             1.3017e-01,  1.4833e+00],
           [ 4.6167e-01,  1.2411e+00,  1.2574e+00,  ...,  3.4412e-01,
             6.1798e+00,  1.2440e+00],
           [ 3.6992e-01,  2.7505e-

In [178]:
%%time 
if use_cuda:
  W.cuda()

#Lamb3 = TV.vSymEig(W.permute((3,4,0,1,2)).reshape((1,9,*W.shape[0:3])), eigenvectors=True)[0].double()
#Wreshape = W.permute((3,4,0,1,2)).reshape((1,9,W.shape[0],W.shape[1],W.shape[2]))
#Lamb3, vec = TV.vSymEig(Wreshape, eigenvectors=True)#[0].double()
vals = TV.vSymEig(W.permute((3,4,0,1,2)).reshape((1,9,*W.shape[0:3])), eigenvectors=True)
Lamb3=torch.abs(vals[0]).double()
#Lamb3 = vals[0].double()
#Lamb3[Lamb3<0]=1e-5
vec=vals[1].double()
#IdxMag = checkMag(Lamb3.reshape((3,*W.shape[0:3])).permute((1,2,3,0)).double(),vec.reshape((3,3,*W.shape[0:3])).permute((2,3,4,0,1)).double())
#IdxMag = checkMag2(Lamb3.reshape((3,*W.shape[0:3])).permute((1,2,3,0)))
#IdxMag = checkMag3(Lamb3.reshape((3,*W.shape[0:3])).permute((1,2,3,0)),W)
IdxMag = checkMag4(Lamb3.reshape((3,*W.shape[0:3])).permute((1,2,3,0)).double(),vec.reshape((3,3,*W.shape[0:3])).permute((2,3,4,0,1)).double())
##smallW = 
if use_cuda:
  Lamb3[0,:,IdxMag[0],IdxMag[1],IdxMag[2]] = torch.symeig(W[IdxMag[0],IdxMag[1],IdxMag[2],:,:].cpu(), eigenvectors=False)[0].permute((1,0)).cuda()
else:
  Lamb3[0,:,IdxMag[0],IdxMag[1],IdxMag[2]] = torch.symeig(W[IdxMag[0],IdxMag[1],IdxMag[2],:,:], eigenvectors=False)[0].permute((1,0))
    
print(len(IdxMag[0]))
##Lamb3[Lamb3<1e-7] = 1e-7
##logLamb3 = torch.log(Lamb3).reshape((3,*W.shape[0:3])).permute((1,2,3,0)).double()
##logLamb3[torch.where(torch.isnan(logLamb3))] = torch.log(Lamb1)[torch.where(torch.isnan(logLamb3))]
trKsquare3 = torch.sum(torch.log(Lamb3)**2,(0,1))
##trKsquare3 = torch.sum(logLamb3**2,(-1))
# CPU performance:
# Just abs: 0 recalc, time = 166ms, trkerror: 0.0204
# checkMag: 85941 recalc, time = 1.08s, trkerror: 0.0155
# checkMag2: 988635 recalc, time = 1.62s, trkerror: 0.0047
# checkMag3: 240 recalc, time = 181ms, trkerror: 0.0239
# checkMag3_v2: 210008 recalc, time = 528ms, trkerror: 0.0002
# GPU performance:
# checkMag: 85617 recalc, time=238ms, trkerror: 0.0069
# checkMag2: 988724 recalc, time=1.58s, trkerror: 0.0083
# checkMag3: 201 recalc, time=112 ms, trkerror: 0.0268
# checkMag3_v2: 210142 recalc, time=458 ms, trkerror: 0.0061

torch.Size([100, 100, 100, 3]) torch.Size([100, 100, 100, 3, 3])
363751 363751 363751
1091253
CPU times: user 1.88 s, sys: 5.94 ms, total: 1.88 s
Wall time: 1.71 s


In [152]:
torch.log(torch.from_numpy(np.array([1.3])))

tensor([0.2624], dtype=torch.float64)

In [305]:
#print(W.shape,Lamb3.shape,len(IdxMag),torch.symeig(W[IdxMag[0],IdxMag[1],IdxMag[2],:,:])[0].shape,smallW.shape)
#print(Lamb3[0,:,IdxMag[0],IdxMag[1],IdxMag[2]].shape)
#print(torch.max(W[IdxMag[0],IdxMag[1],IdxMag[2],:,:]))
print(len(IdxMag[0][0]))

28814


In [122]:
%%time
Lamb3Scaled = TV.vSymEig(scaled_W.permute((3,4,0,1,2)).reshape((1,9,*W.shape[0:3])), eigenvectors=False)[0].reshape((3,*W.shape[0:3])).permute((1,2,3,0))
Lamb3Alt = torch.einsum('...,...i->...i', W_max, Lamb3Scaled)
trKsquare3Alt = torch.sum(torch.log(Lamb3Alt)**2,(-1))

CPU times: user 960 ms, sys: 13.3 ms, total: 973 ms
Wall time: 82.2 ms


In [128]:
print(W_max.shape)
print(scaled_W.shape)
print(torch.min(Lamb3Alt))


torch.Size([100, 100, 100])
torch.Size([100, 100, 100, 3, 3])
tensor(-4757.7923, dtype=torch.float64)


 Method 4: NISymmetricEigensolver3x3

In [26]:
%%time
Lamb4 = NISymmetricEigensolver3x3(W)
trKsquare4 = torch.sum(torch.log(Lamb4)**2,(-1))

RuntimeError: CUDA error: device-side assert triggered
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.

### Check the diff 

In [18]:
def checkNaN(A):
    if (A != A).any():
        print('NaN')
        return(np.argwhere(np.isnan(A.numpy())))
    else:
        print('Good')

def check_L2_diff_error(a,b):
    L2Diff = LA.norm(a - b)
    L2Error =  2*LA.norm(a - b)/( LA.norm(a)+LA.norm(b))
    return L2Diff, L2Error

In [112]:
# check diff11alt
check_L2_diff_error(trKsquare1,trKsquare1Alt)

(tensor(13.3727, dtype=torch.float64), tensor(0.0003, dtype=torch.float64))

In [24]:
# check diff12
check_L2_diff_error(trKsquare1,trKsquare2)

(tensor(6.4856e-13, dtype=torch.float64),
 tensor(1.6858e-16, dtype=torch.float64))

In [161]:
# check diff13
check_L2_diff_error(trKsquare1,trKsquare3)

(tensor(113.0688, device='cuda:0', dtype=torch.float64),
 tensor(0.0022, device='cuda:0', dtype=torch.float64))

In [61]:
# check diff23
check_L2_diff_error(trKsquare2,trKsquare3)

NameError: name 'trKsquare2' is not defined

In [62]:
# check diff23
check_L2_diff_error(trKsquare2,trKsquare3Alt)

NameError: name 'trKsquare2' is not defined

In [148]:
# check diff14
check_L2_diff_error(trKsquare1,trKsquare4)

(tensor(nan, dtype=torch.float64), tensor(nan, dtype=torch.float64))

In [141]:
print(Lamb4.shape)
print(trKsquare4.shape)

torch.Size([3, 100, 100, 100])
torch.Size([3, 100, 100])


### Check if there are NaN values in trKsquare3

In [15]:
checkNaN(trKsquare1),checkNaN(trKsquare2),checkNaN(trKsquare3)

Good
Good
NaN


(None,
 None,
 array([[ 0, 58, 40],
        [ 1, 16, 52],
        [ 1, 26, 98],
        [ 1, 72, 12],
        [ 1, 86, 20],
        [ 2, 23, 26],
        [ 2, 32,  7],
        [ 2, 45, 14],
        [ 2, 96, 96],
        [ 3, 11, 25],
        [ 3, 22, 34],
        [ 3, 31, 87],
        [ 3, 99, 70],
        [ 4, 45, 63],
        [ 5, 58, 52],
        [ 6, 47, 97],
        [ 6, 77, 47],
        [ 6, 91, 12],
        [ 7, 77, 57],
        [ 7, 77, 83],
        [ 7, 82,  2],
        [ 8, 15, 68],
        [ 8, 23, 73],
        [ 8, 55, 40],
        [ 8, 99, 65],
        [ 9, 58,  5],
        [ 9, 94, 49],
        [ 9, 99, 16],
        [10, 39, 69],
        [10, 96,  3],
        [11, 21, 28],
        [11, 28, 47],
        [12, 58, 74],
        [13, 51, 37],
        [13, 99, 92],
        [14, 27, 34],
        [14, 28, 10],
        [14, 52,  7],
        [14, 72, 78],
        [15, 35, 39],
        [15, 80, 91],
        [16, 15, 37],
        [17,  7, 66],
        [17, 69, 14],
        [17, 98, 1

In [16]:
# Check if there are NaN values in Lamb
checkNaN(Lamb1), checkNaN(Lamb2), checkNaN(Lamb3)

Good
Good
Good


(None, None, None)

# Check NaN Eigenvectors

In [31]:
checkNaN(Lamb1v), checkNaN(Lamb2v)

Good
NaN


(None,
 array([[ 83613,      0,      0],
        [ 83613,      0,      1],
        [ 83613,      0,      2],
        [ 83625,      0,      0],
        [ 83625,      0,      1],
        [ 83625,      0,      2],
        [ 88047,      0,      0],
        [ 88047,      0,      1],
        [ 88047,      0,      2],
        [ 88129,      0,      0],
        [ 88129,      0,      1],
        [ 88129,      0,      2],
        [ 91978,      0,      0],
        [ 91978,      0,      1],
        [ 91978,      0,      2],
        [ 91988,      0,      0],
        [ 91988,      0,      1],
        [ 91988,      0,      2],
        [104444,      0,      0],
        [104444,      0,      1],
        [104444,      0,      2],
        [104450,      0,      0],
        [104450,      0,      1],
        [104450,      0,      2],
        [104693,      0,      0],
        [104693,      0,      1],
        [104693,      0,      2],
        [141591,      0,      0],
        [141591,      0,      1],
       

In [33]:
idx=83613
print(Lamb1[idx])
print(Lamb1v[idx])
print(Lamb2[idx])
print(Lamb2v[idx])

tensor([0.0167, 0.1000, 0.1000], dtype=torch.float64)
tensor([[-0.5150, -0.8572,  0.0000],
        [-0.8572,  0.5150,  0.0000],
        [-0.0000,  0.0000,  1.0000]], dtype=torch.float64)
tensor([0.1000, 0.1000, 0.0167], dtype=torch.float64)
tensor([[    nan,     nan,     nan],
        [ 0.0000,  0.0000, -1.0000],
        [-0.5150, -0.8572,  0.0000]], dtype=torch.float64)


In [77]:
Lamb2v[idx]

tensor([[    nan,     nan,     nan],
        [ 0.0000,  0.0000, -1.0000],
        [-0.5150, -0.8572,  0.0000]], dtype=torch.float64)

In [74]:
print(Lamb1[idx,2],Lamb1v[idx,2,:])
print(Lamb2[idx,1],Lamb2v[idx,1,:])

tensor(0.1000, dtype=torch.float64) tensor([-0., 0., 1.], dtype=torch.float64)
tensor(0.1000, dtype=torch.float64) tensor([ 0.,  0., -1.], dtype=torch.float64)


In [69]:
AlI = W[idx] - Lamb2[idx,1] * torch.eye(3)
print(AlI)
print(AlI[:,0])

tensor([[-2.2102e-02, -3.6788e-02,  0.0000e+00],
        [-3.6788e-02, -6.1231e-02,  0.0000e+00],
        [ 0.0000e+00,  0.0000e+00, -1.4901e-09]], dtype=torch.float64)
tensor([-0.0221, -0.0368,  0.0000], dtype=torch.float64)


In [70]:
Ale = W[idx] - Lamb2[idx,1] * Lamb2v[idx,1,:]
print(Ale)
print(Ale[:,0])

tensor([[ 0.0779, -0.0368,  0.1000],
        [-0.0368,  0.0388,  0.1000],
        [ 0.0000,  0.0000,  0.2000]], dtype=torch.float64)
tensor([ 0.0779, -0.0368,  0.0000], dtype=torch.float64)


In [76]:
newvI=torch.cross(AlI[:,0],Lamb2v[idx,1,:])
newvI = newvI / torch.linalg.norm(newvI)
print(newvI)

tensor([ 0.8572, -0.5150,  0.0000], dtype=torch.float64)


In [72]:
newve=torch.cross(Ale[:,0],Lamb2v[idx,1,:])
newve = newve / torch.linalg.norm(newve)
print(newve)

tensor([0.4270, 0.9042, 0.0000], dtype=torch.float64)


In [79]:
v = torch.from_numpy(np.array([0.0466, -0.0414,0]))
normv = v / torch.linalg.norm(v)
print(normv)

tensor([ 0.7476, -0.6642,  0.0000], dtype=torch.float64)


### Compute the index of a NaN value 

In [17]:
IndNaN = checkNaN(trKsquare3)
print(len(IndNaN))
a,b,c = IndNaN[0]
a,b,c

NaN
203


(0, 58, 40)

### Check the PDS matrix at the index

In [18]:
IndNaN = checkNaN(trKsquare3)
test = W[a,b,c]

# Compare the entries in W and reshaped W
print(W[a,b,c])
W.permute((3,4,0,1,2)).reshape((1,9,*W.shape[0:3]))[:,:,a,b,c]

NaN
tensor([[ 3.2045e+00,  8.4586e-02, -4.9083e+04],
        [ 8.4586e-02,  1.9889e+00, -4.1348e+04],
        [-4.9083e+04, -4.1348e+04,  1.5827e+09]], dtype=torch.float64)


tensor([[ 3.2045e+00,  8.4586e-02, -4.9083e+04,  8.4586e-02,  1.9889e+00,
         -4.1348e+04, -4.9083e+04, -4.1348e+04,  1.5827e+09]],
       dtype=torch.float64)

In [19]:
# check if the matrix is symetric
test - test.t()

tensor([[ 0.0000e+00, -7.3552e-16, -7.2760e-12],
        [ 7.3552e-16,  0.0000e+00, -7.2760e-12],
        [ 7.2760e-12,  7.2760e-12,  0.0000e+00]], dtype=torch.float64)

In [20]:
torch.det(test)

tensor(1.4917e+08, dtype=torch.float64)

In [21]:
# check if the matrix is pd
torch.symeig(test)[0]

tensor([3.6900e-02, 2.5541e+00, 1.5827e+09], dtype=torch.float64)

In [22]:
# Compare the eigenvalues 

# compute the eigenvalues using vSymEig
eigTest = TV.vSymEig(test.reshape((1,9,1,1,1)), eigenvectors=False)[0]
eigTest

tensor([[[[[-5.1234e+00]]],


         [[[ 7.7144e+00]]],


         [[[ 1.5827e+09]]]]])

In [31]:
symtest = (test  + test.t())/2
symtest-symtest.t()

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]], dtype=torch.float64)

In [32]:
torch.symeig(symtest)[0]

tensor([3.6900e-02, 2.5541e+00, 1.5827e+09], dtype=torch.float64)

In [33]:
eigTest = TV.vSymEig(symtest.reshape((1,9,1,1,1)), eigenvectors=False)[0]
eigTest

tensor([[[[[-5.1234e+00]]],


         [[[ 7.7144e+00]]],


         [[[ 1.5827e+09]]]]])

### Check the eigenvalues computed previously

In [23]:
print(Lamb1[a,b,c])
print(Lamb1[a,b,c])
print(Lamb3[:,:,a,b,c])

tensor([3.6900e-02, 2.5541e+00, 1.5827e+09], dtype=torch.float64)
tensor([3.6900e-02, 2.5541e+00, 1.5827e+09], dtype=torch.float64)
tensor([[-5.1234e+00,  7.7144e+00,  1.5827e+09]])


In [24]:
min_max = torch.max(Lamb1[a,b,c])
min_max_idx = IndNaN[0]
max_max = torch.max(Lamb1[a,b,c])
max_max_idx = IndNaN[0]
for idx in IndNaN:
  max_eval = torch.max(Lamb1[idx[0],idx[1],idx[2]])
  print(max_eval)
  if max_eval < min_max:
    min_max = max_eval
    min_max_idx = idx
  if max_eval > max_max:
    max_max = max_eval
    max_max_idx = idx
print(min_max, max_max)
#  print(torch.max(Lamb3[:,:,idx[0],idx[1],idx[2]]))  

tensor(1.5827e+09, dtype=torch.float64)
tensor(99630454.1432, dtype=torch.float64)
tensor(23692036.4034, dtype=torch.float64)
tensor(1301049.0521, dtype=torch.float64)
tensor(584963.1336, dtype=torch.float64)
tensor(34745428.6724, dtype=torch.float64)
tensor(1547995.9492, dtype=torch.float64)
tensor(1451792.2938, dtype=torch.float64)
tensor(19738185.0698, dtype=torch.float64)
tensor(2.3989e+08, dtype=torch.float64)
tensor(3.5612e+08, dtype=torch.float64)
tensor(7272763.7192, dtype=torch.float64)
tensor(1.4080e+08, dtype=torch.float64)
tensor(5.9377e+08, dtype=torch.float64)
tensor(4600587.4629, dtype=torch.float64)
tensor(2.2020e+08, dtype=torch.float64)
tensor(5.4657e+08, dtype=torch.float64)
tensor(1.0701e+08, dtype=torch.float64)
tensor(18485300.0272, dtype=torch.float64)
tensor(2.8805e+09, dtype=torch.float64)
tensor(56554163.8674, dtype=torch.float64)
tensor(2.2583e+08, dtype=torch.float64)
tensor(29901546.3968, dtype=torch.float64)
tensor(16539309.6501, dtype=torch.float64)
tenso

In [25]:
min_max_idx

array([63, 62,  0])

In [26]:
d,e,f = min_max_idx
test2 = W[d,e,f]

# Compare the entries in W and reshaped W
print(W[d,e,f])
W.permute((3,4,0,1,2)).reshape((1,9,*W.shape[0:3]))[:,:,d,e,f]

tensor([[   16.2573,    52.6822,   422.1447],
        [   52.6822,   175.2501,  1430.2641],
        [  422.1447,  1430.2641, 11817.8356]], dtype=torch.float64)


tensor([[   16.2573,    52.6822,   422.1447,    52.6822,   175.2501,  1430.2641,
           422.1447,  1430.2641, 11817.8356]], dtype=torch.float64)

In [29]:
# check if the matrix is symetric
test2 - test2.t()

tensor([[ 0.0000e+00,  7.1054e-15,  1.1369e-13],
        [-7.1054e-15,  0.0000e+00, -2.2737e-13],
        [-1.1369e-13,  2.2737e-13,  0.0000e+00]], dtype=torch.float64)

In [30]:
torch.det(test2)

tensor(8.9804e-06, dtype=torch.float64)

In [34]:
# check if the matrix is pd
torch.symeig(test2)[0]

tensor([2.2786e-10, 3.2829e+00, 1.2006e+04], dtype=torch.float64)

In [35]:
eigTest2 = TV.vSymEig(test2.reshape((1,9,1,1,1)), eigenvectors=False)[0]
eigTest2

tensor([[[[[-1.4993e-09]]],


         [[[ 3.2829e+00]]],


         [[[ 1.2006e+04]]]]])

In [46]:
eigTest2, evtest2 = TV.vSymEig(test2.reshape((1,9,1,1,1)), eigenvectors=True)

In [38]:
eigTest2[eigTest2<0] = 1.0e-7

In [49]:
recontest2 = torch.einsum('...ji,...j,...ik->...ik',evtest2,eigTest2,evtest2)
recontest2.reshape((3,3))

tensor([[-9.6699e-10, -5.3043e-10, -1.8852e-12],
        [ 1.1594e+00,  2.0761e+00,  4.7352e-02],
        [ 2.2444e+01,  1.6583e+02,  1.1818e+04]])

In [43]:
test2

tensor([[   16.2573,    52.6822,   422.1447],
        [   52.6822,   175.2501,  1430.2641],
        [  422.1447,  1430.2641, 11817.8356]], dtype=torch.float64)

In [44]:
print(recontest2.shape,test2.shape)

torch.Size([1, 3, 3, 1, 1, 1]) torch.Size([3, 3])


In [61]:
torch.einsum('...ij,...j->...i',recontest2.reshape((3,3)),evtest2.reshape((3,3))[0,:])

tensor([-1.0921e-09,  2.1677e+00,  5.3571e+02])

In [62]:
1.e-7 * evtest2.reshape((3,3))[0,:]

tensor([8.0309e-08, 5.9480e-08, 3.5459e-09])

In [25]:
# From https://www.geometrictools.com/GTE/Mathematics/SymmetricEigensolver3x3.h
def NISymmetricEigensolver3x3(A):
 # The input matrix must be symmetric, so only the unique elements
 # must be specified: a00, a01, a02, a11, a12, and a22.  The
 # eigenvalues are sorted in ascending order: eval0 <= eval1 <= eval2.

  #Precondition the matrix by factoring out the maximum absolute
  # value of the components.  This guards against floating-point
  # overflow when computing the eigenvalues.
  A_max = torch.max(A.reshape((*A.shape[0:3],9)),3)[0]
  scaled_A = torch.einsum('...ij,...->...ij',A, 1.0 / A_max)
  #          Real max0 = std::max(std::fabs(a00), std::fabs(a01));
  #          Real max1 = std::max(std::fabs(a02), std::fabs(a11));
  #          Real max2 = std::max(std::fabs(a12), std::fabs(a22));
  #          Real maxAbsElement = std::max(std::max(max0, max1), max2);
  if torch.any(A_max < 1e-12):
    print ('small A_max detected')
    #        if (maxAbsElement == (Real)0)
    #        {
    #            // A is the zero matrix.
    #            eval[0] = (Real)0;
    #            eval[1] = (Real)0;
    #            eval[2] = (Real)0;
    #            evec[0] = { (Real)1, (Real)0, (Real)0 };
    #            evec[1] = { (Real)0, (Real)1, (Real)0 };
    #            evec[2] = { (Real)0, (Real)0, (Real)1 };
    #            return;
    #        }

    #        Real invMaxAbsElement = (Real)1 / maxAbsElement;
    #        a00 *= invMaxAbsElement;
    #        a01 *= invMaxAbsElement;
    #        a02 *= invMaxAbsElement;
    #        a11 *= invMaxAbsElement;
    #        a12 *= invMaxAbsElement;
    #        a22 *= invMaxAbsElement;

  a01 = scaled_A[:,:,:,0,1]  
  a02 = scaled_A[:,:,:,0,2]
  a12 = scaled_A[:,:,:,1,2]
  nrm = a01 * a01 + a02 * a02 + a12 * a12
  #        Real norm = a01 * a01 + a02 * a02 + a12 * a12;
  if torch.any(nrm < 1e-12):
    print('Need diagonal case')
                #// The matrix is diagonal.
                #eval[0] = a00;
                #eval[1] = a11;
                #eval[2] = a22;
                #evec[0] = { (Real)1, (Real)0, (Real)0 };
                #evec[1] = { (Real)0, (Real)1, (Real)0 };
                #evec[2] = { (Real)0, (Real)0, (Real)1 };
  else:
      #          // Compute the eigenvalues of A.

      #          // In the PDF mentioned previously, B = (A - q*I)/p, where
      #          // q = tr(A)/3 with tr(A) the trace of A (sum of the diagonal
      #          // entries of A) and where p = sqrt(tr((A - q*I)^2)/6).
      #          Real q = (a00 + a11 + a22) / (Real)3;
    q = (scaled_A[:,:,:,0,0] + scaled_A[:,:,:,1,1] + scaled_A[:,:,:,2,2]) / 3.0

      #          // The matrix A - q*I is represented by the following, where
      #          // b00, b11 and b22 are computed after these comments,
      #          //   +-           -+
      #          //   | b00 a01 a02 |
      #          //   | a01 b11 a12 |
      #          //   | a02 a12 b22 |
      #          //   +-           -+
      #          Real b00 = a00 - q;
      #          Real b11 = a11 - q;
      #          Real b22 = a22 - q;
    b00 = scaled_A[:,:,:,0,0] - q
    b11 = scaled_A[:,:,:,1,1] - q
    b22 = scaled_A[:,:,:,2,2] - q

      #          // The is the variable p mentioned in the PDF.
      #          Real p = std::sqrt((b00 * b00 + b11 * b11 + b22 * b22 + norm * (Real)2) / (Real)6);
    p = torch.sqrt((b00 * b00 + b11 * b11 + b22 * b22 + nrm * 2.0) / 6.0)
      #          // We need det(B) = det((A - q*I)/p) = det(A - q*I)/p^3.  The
      #          // value det(A - q*I) is computed using a cofactor expansion
      #          // by the first row of A - q*I.  The cofactors are c00, c01
      #          // and c02 and the determinant is b00*c00 - a01*c01 + a02*c02.
      #          // The det(B) is then computed finally by the division
      #          // with p^3.
      #          Real c00 = b11 * b22 - a12 * a12;
      #          Real c01 = a01 * b22 - a12 * a02;
      #          Real c02 = a01 * a12 - b11 * a02;
      #          Real det = (b00 * c00 - a01 * c01 + a02 * c02) / (p * p * p);
    c00 = b11 * b22 - a12 * a12
    c01 = a01 * b22 - a12 * a02
    c02 = a01 * a12 - b11 * a02
    det = (b00 * c00 - a01 * c01 + a02 * c02) / (p * p * p)

      #          // The halfDet value is cos(3*theta) mentioned in the PDF. The
      #          // acos(z) function requires |z| <= 1, but will fail silently
      #          // and return NaN if the input is larger than 1 in magnitude.
      #          // To avoid this problem due to rounding errors, the halfDet
      #          // value is clamped to [-1,1].
      #          Real halfDet = det * (Real)0.5;
      #          halfDet = std::min(std::max(halfDet, (Real)-1), (Real)1);
    halfDet = 0.5 * det
    halfDet[halfDet < -1] = -1
    halfDet[halfDet > 1] = 1

      #          // The eigenvalues of B are ordered as
      #          // beta0 <= beta1 <= beta2.  The number of digits in
      #          // twoThirdsPi is chosen so that, whether float or double,
      #          // the floating-point number is the closest to theoretical
      #          // 2*pi/3.
      #          Real angle = std::acos(halfDet) / (Real)3;
      #          Real const twoThirdsPi = (Real)2.09439510239319549;
      #          Real beta2 = std::cos(angle) * (Real)2;
      #          Real beta0 = std::cos(angle + twoThirdsPi) * (Real)2;
      #          Real beta1 = -(beta0 + beta2);
    angle = torch.acos(halfDet) / 3.0
    twoThirdsPi = 2.09439510239319549
    beta2 = torch.cos(angle) * 2.0
    beta0 = torch.cos(angle + twoThirdsPi) * 2.0
    beta1 = -(beta0 + beta2)

      #          // The eigenvalues of A are ordered as
      #          // alpha0 <= alpha1 <= alpha2.
      #          eval[0] = q + p * beta0;
      #          eval[1] = q + p * beta1;
      #          eval[2] = q + p * beta2;
    eval0 = q + p * beta0
    eval1 = q + p * beta1
    eval2 = q + p * beta2
    

      #          // Compute the eigenvectors so that the set
      #          // {evec[0], evec[1], evec[2]} is right handed and
      #          // orthonormal.
      #          if (halfDet >= (Real)0)
      #          {
      #              ComputeEigenvector0(a00, a01, a02, a11, a12, a22, eval[2], evec[2]);
      #              ComputeEigenvector1(a00, a01, a02, a11, a12, a22, evec[2], eval[1], evec[1]);
      #              evec[0] = Cross(evec[1], evec[2]);
      #          }
      #          else
      #          {
      #              ComputeEigenvector0(a00, a01, a02, a11, a12, a22, eval[0], evec[0]);
      #              ComputeEigenvector1(a00, a01, a02, a11, a12, a22, evec[0], eval[1], evec[1]);
      #              evec[2] = Cross(evec[0], evec[1]);
      #          }
      #      }
  print('Skipping eigenvector computation for now')

      #      // The preconditioning scaled the matrix A, which scales the
      #      // eigenvalues.  Revert the scaling.
      #      eval[0] *= maxAbsElement;
      #      eval[1] *= maxAbsElement;
      #      eval[2] *= maxAbsElement;
  evals = torch.stack((eval0 * A_max, eval1 * A_max, eval2 * A_max),3) 
  print(eval0.shape, evals.shape)

  #          SortEigenstuff<Real>()(sortType, true, eval, evec);
  return(torch.sort(evals,-1)[0])